# Treinamento do modelo final

Até aqui, todos os nossos desenvolvimentos se basearam em treinar o modelo em uma base de treino, tendo uma base de validação para avaliar o modelo ao final de tudo. Nesse momento, onde já temos todas as definições em relação à modelagem, vamos treinar o modelo com a configuração escolhida em toda a base disponível.

In [1]:
#carregando as bibliotecas necessárias
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import f1_score, recall_score, precision_score
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold, cross_val_score

import FeatureGenerator as fg

from joblib import dump, load

from lightgbm import LGBMClassifier

In [2]:
#carregando ambas as bases que temos disponíveis
df_train = pd.read_csv("../inputs/train.csv")
df_val = pd.read_csv("../inputs/validation.csv")

In [3]:
#concatenando as partições para termos a base completa
data = pd.concat([df_train, df_val])

In [4]:
#o lightgbm irá converter para NaN todas as variáveis categóricas que forem menor que 0
#para esses casos, teremos que aplicar um tratamento diferente
#vamos escolher um valor qualquer negativo para multiplicar as categorias negativas e criar novas categorias positivas
def negative_cat(value):
    if value < 0:
        value = value*(-15)
        
    else:
        pass
    
    return value

In [5]:
#selecionando as variáveis categóricas para definir no modelo
cat_cols = ['SEX',
 'EDUCATION',
 'MARRIAGE',
 'PAY_0',
 'PAY_2',
 'PAY_3',
 'PAY_4',
 'PAY_5',
 'PAY_6']

In [6]:
#aplicando a função
for c in cat_cols:
    data[c] = data[c].apply(negative_cat)

In [7]:
#criando input e output
X, y = data.drop(columns='target', axis=1), data.target.values

In [8]:
#o lightgbm só aceita index para identificação das colunas categoricas
cat_idx = []
for c in cat_cols:
    #aqui estamos subtraindo 1 do index porque a coluna "ID" será removida, alterando o index das colunas restantes
    idx = X.columns.get_loc(c) - 1
    cat_idx.append(idx)

In [9]:
#inicializando as classes que serão utilizadas para geração de features
#a ideia de utilizar classes é poder encapsular tudo num só pipeline

#classe para criar as features de divisão
fbf = fg.FeatureByFeature(features_num=["PAY_AMT1", "PAY_AMT2", "PAY_AMT3", "PAY_AMT4", "PAY_AMT5", "PAY_AMT6"],
                         features_denom=["BILL_AMT1", "BILL_AMT2", "BILL_AMT3", "BILL_AMT4", "BILL_AMT5", "BILL_AMT6"])

#classe para criar as features de diferença
diff1 = fg.DiffFeatures(features=["PAY_AMT1", "PAY_AMT2", "PAY_AMT3", "PAY_AMT4", "PAY_AMT5", "PAY_AMT6"])

diff2 = fg.DiffFeatures(features=["BILL_AMT1", "BILL_AMT2", "BILL_AMT3", "BILL_AMT4", "BILL_AMT5", "BILL_AMT6"])

#classe para dropar features que não serão necessárias
dc = fg.DropCols(features=["ID"])

In [10]:
#criando nosso dicionário de parâmetros para configurar o modelo
selected_params = {'reg_lambda': 4,
 'reg_alpha': 12,
 'num_leaves': 12,
 'n_estimators': 500,
 'min_child_samples': 60,
 'max_depth': 8,
 'learning_rate': 0.005,
 'colsample_bytree': None}

In [11]:
#inicializando um novo lightgbm, agora com os parâmetros escolhidos
estimator = LGBMClassifier(**selected_params,
                           categorical_features=cat_idx,
                           class_weight="balanced",
                           random_state=42)

In [12]:
#configurando o pipeline com o novo modelo inicializado
pipe = Pipeline(steps=[("DropCols", dc),
                       ("FeatureByFeature", fbf),
                      ("diff_features1", diff1),
                      ("diff_features2", diff2),
                      ("Estimator", estimator)])

In [13]:
#fit
pipe.fit(X, y)

Pipeline(steps=[('DropCols', DropCols(features=['ID'])),
                ('FeatureByFeature',
                 FeatureByFeature(features_denom=['BILL_AMT1', 'BILL_AMT2',
                                                  'BILL_AMT3', 'BILL_AMT4',
                                                  'BILL_AMT5', 'BILL_AMT6'],
                                  features_num=['PAY_AMT1', 'PAY_AMT2',
                                                'PAY_AMT3', 'PAY_AMT4',
                                                'PAY_AMT5', 'PAY_AMT6'])),
                ('diff_features1',
                 DiffFeatures(features=['PAY_AMT1', 'PAY_AMT2', 'PAY_AMT3',
                                        'PAY_AMT4', '...
                ('diff_features2',
                 DiffFeatures(features=['BILL_AMT1', 'BILL_AMT2', 'BILL_AMT3',
                                        'BILL_AMT4', 'BILL_AMT5',
                                        'BILL_AMT6'])),
                ('Estimator',
                 LGBMCla

In [14]:
#salvando nosso pipeline inteiro em .pkl
dump(pipe, "../app/model.pkl")

['../app/model.pkl']

# Observações finais 

Note que todas as nossas transformações, mesmo as customizadas para feature engineering, foram encapsuladas no pipeline do modelo e estarão salvas no nosso arquivo .pkl, o que tornará o deploy do nosso modelo muito mais "clean" em termos de código.